In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import gzip
from datetime import datetime, timedelta
import numpy as np
from statistics import mean
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
def retrieve_data(filename, varname):
    df_new = pd.read_csv(filename)
    df_new["Date"] = pd.to_datetime(df_new["Date"])
    try:
        df_new.columns = ["Date", "Close", "Open", "High", "Low", "Vol.", "Change%"]
    except ValueError:
        try:
            df_new.columns = ["Date", "Close", "Open", "High", "Low", "Change%"]
        except ValueError:
            df_new.columns = ["Date", "Close", "Open", "High", "Low"]

    rows = []
    for index, row in df_new.iterrows():
        date = row['Date']
        p_close = float(str(row['Close']).replace(',', ''))
        p_open = float(str(row['Open']).replace(',', ''))
        p_high = float(str(row['High']).replace(',', ''))
        p_low = float(str(row['Low']).replace(',', ''))
        if df_new.columns.tolist() == ["Date", "Close", "Open", "High", "Low", "Vol.", "Change%"]:
            if str(row['Vol.']) == "-":
                volume = 0
            else:
                if row['Vol.'][-1] == "B": # volume in billions
                    volume = float(row['Vol.'][:len(row['Vol.']) - 1]) * 1000000000
                elif row['Vol.'][-1] == "M": # volume in millions
                    volume = float(row['Vol.'][:len(row['Vol.']) - 1]) * 1000000
                elif row['Vol.'][-1] == "K": # volume in thousands
                    volume = float(row['Vol.'][:len(row['Vol.']) - 1]) * 1000
                else:
                    print(varname, row['Date'], row['Vol.'])
            rows.append([date, p_close, p_open, p_high, p_low, volume])
        else:
            rows.append([date, p_close, p_open, p_high, p_low])
    df_new = pd.DataFrame(rows)
    if len(rows[0]) == 6:
        df_new.columns = ["Date", varname + "_Close", varname + "_Open", varname + "_High", varname + "_Low", varname + "_Volume"]
    else:
        df_new.columns = ["Date", varname + "_Close", varname + "_Open", varname + "_High", varname + "_Low"]
    
    df_new = df_new.sort_values(by='Date').reset_index(drop=True)
#     plot_closes_crises(df_new, "Plot of " + varname + " Daily Close Prices", varname + "_Close", varname)
    
#     perc_crash_2008 = crisis_analysis(df_new, datetime(2007, 10, 1), datetime(2009, 6, 1), varname)
#     perc_crash_covid = crisis_analysis(df_new, datetime(2020, 2, 1), datetime(2020, 12, 1), varname)
#     print(f"{varname} 2008 crash {perc_crash_2008}")
#     print(f"{varname} Covid crash {perc_crash_covid}")
   
    # remove crises
    df_new = df_new[(df_new['Date'] >= datetime(2009, 7, 1)) & (df_new['Date'] <= datetime(2019, 12, 31))]
    
    return df_new

files = {
    # varname: filename
    "S&P500": "Dataset v2/Indices/S&P 500 Historical Data.csv",
    "US30": "Dataset v2/Indices/Dow Jones Industrial Average Historical Data.csv", 
    "US2000": "Dataset v2/Indices/US Small Cap 2000 Historical Data.csv",
    "NASDAQ": "Dataset v2/Indices/NASDAQ Composite Historical Data.csv", 
    "GER30": "Dataset v2/Indices/DAX Historical Data.csv",
    "CAC40": "Dataset v2/Indices/CAC 40 Historical Data.csv",
    "UK100": "Dataset v2/Indices/invUK100 Historical Data.csv",
    "SSE50": "Dataset v2/Indices/Shanghai Composite Historical Data.csv",
    "HS50": "Dataset v2/Indices/Hang Seng Historical Data.csv",
    "NIKKEI225": "Dataset v2/Indices/NIKKEI 225 Historical Data.csv",
}

print(f"Retrieving {len(files)} files")
i = 1
df = pd.DataFrame()
stats_rows = []
for file in files:
    df = retrieve_data(files[file], file)
    if file == "S&P500":
        display(df)
    print(f"{file} number of trading days: {len(df)}")
    i += 1

Retrieving 10 files


,Date,S&P500_Close,S&P500_Open,S&P500_High,S&P500_Low,S&P500_Volume
1131,2009-07-01,923.33,920.82,931.92,920.82,0
1132,2009-07-02,896.42,921.24,921.24,896.42,0
1133,2009-07-06,898.72,894.27,898.72,886.36,0
1134,2009-07-07,881.03,898.60,898.60,879.93,0
1135,2009-07-08,879.56,881.90,886.80,869.32,0
1136,2009-07-09,882.68,881.28,887.86,878.45,0
1137,2009-07-10,879.13,880.03,883.57,872.81,0
1138,2009-07-13,901.05,879.57,901.05,875.32,0
1139,2009-07-14,905.84,900.77,905.84,896.50,0
1140,2009-07-15,932.68,910.15,933.95,910.15,0


S&P500 number of trading days: 2644
US30 number of trading days: 2644
US2000 number of trading days: 2644
NASDAQ number of trading days: 2644
GER30 number of trading days: 2665
CAC40 number of trading days: 2690
UK100 number of trading days: 2653
SSE50 number of trading days: 2558
HS50 number of trading days: 2597
NIKKEI225 number of trading days: 2599


In [110]:
# US holidays as of july 2009, seems to exclude half days
# half days will have prices, but with shorter opening times, there may be less trades, less volatility
US_holidays = [
            datetime(2009,7,3), # Independence day
            datetime(2009,9,7), # Labor day
            datetime(2009,11,26), # Thanksgiving
            datetime(2009,12,25), # Christmas
               
            datetime(2010,1,1), # New Year's day
            datetime(2010,1,18), # Martin Luther King Jr. day
            datetime(2010,2,15), # President's day
            datetime(2010,4,2), # Good Friday
            datetime(2010,5,31), # Memorial day
            datetime(2010,7,5), # Independence day
            datetime(2010,9,6), # Labor day
            datetime(2010,11,25), # Thanksgiving
            datetime(2010,12,24), # Christmas
               
            datetime(2011,1,1), # New Year's day
            datetime(2011,1,17), # Martin Luther King Jr. day
            datetime(2011,2,21), # President's day
            datetime(2011,4,22), # Good Friday
            datetime(2011,5,30), # Memorial day
            datetime(2011,7,4), # Independence day
            datetime(2011,9,5), # Labor day
            datetime(2011,11,24), # Thanksgiving
            datetime(2011,12,26), # Christmas
               
            datetime(2012,1,1), # New Year's day
            datetime(2012,1,2), # New Year's day
            datetime(2012,1,16), # Martin Luther King Jr. day
            datetime(2012,2,20), # President's day
            datetime(2012,4,6), # Good Friday
            datetime(2012,5,28), # Memorial day
            datetime(2012,7,4), # Independence day
            datetime(2012,9,3), # Labor day
            datetime(2012,11,22), # Thanksgiving
            datetime(2012,12,25), # Christmas

            datetime(2013,1,1), # New Year's day
            datetime(2013,1,21), # Martin Luther King Jr. day
            datetime(2013,2,18), # President's day
            datetime(2013,3,29), # Good Friday
            datetime(2013,5,27), # Memorial day
            datetime(2013,7,4), # Independence day
            datetime(2013,9,2), # Labor day
            datetime(2013,11,28), # Thanksgiving
            datetime(2013,12,25), # Christmas
            
            datetime(2014,1,1), # New Year's day
            datetime(2014,1,20), # Martin Luther King Jr. day
            datetime(2014,2,17), # President's day
            datetime(2014,4,18), # Good Friday
            datetime(2014,5,26), # Memorial day
            datetime(2014,7,4), # Independence day
            datetime(2014,9,1), # Labor day
            datetime(2014,11,27), # Thanksgiving
            datetime(2014,12,25), # Christmas
            
            datetime(2015,1,1), # New Year's day
            datetime(2015,1,19), # Martin Luther King Jr. day
            datetime(2015,2,16), # President's day
            datetime(2015,4,3), # Good Friday
            datetime(2015,5,25), # Memorial day
            datetime(2015,7,3), # Independence day
            datetime(2015,9,7), # Labor day
            datetime(2015,11,26), # Thanksgiving
            datetime(2015,12,25), # Christmas
                     
            datetime(2016,1,1), # New Year's day
            datetime(2016,1,18), # Martin Luther King Jr. day
            datetime(2016,2,15), # President's day
            datetime(2016,3,25), # Good Friday
            datetime(2016,5,30), # Memorial day
            datetime(2016,7,4), # Independence day
            datetime(2016,9,5), # Labor day
            datetime(2016,11,24), # Thanksgiving
            datetime(2016,12,26), # Christmas
            
            datetime(2017,1,1), # New Year's day
            datetime(2017,1,2), # New Year's day
            datetime(2017,1,16), # Martin Luther King Jr. day
            datetime(2017,2,20), # President's day
            datetime(2017,4,14), # Good Friday
            datetime(2017,5,29), # Memorial day
            datetime(2017,7,4), # Independence day
            datetime(2017,9,4), # Labor day
            datetime(2017,11,23), # Thanksgiving
            datetime(2017,12,25), # Christmas
               
            datetime(2018,1,1), # New Year's day
            datetime(2018,1,15), # Martin Luther King Jr. day
            datetime(2018,2,19), # President's day
            datetime(2018,3,30), # Good Friday
            datetime(2018,5,28), # Memorial day
            datetime(2018,7,4), # Independence day
            datetime(2018,9,3), # Labor day
            datetime(2018,11,22), # Thanksgiving
            datetime(2018,12,5), # Christmas
            datetime(2018,12,25), # Christmas
            
            datetime(2019,1,1), # New Year's day
            datetime(2019,1,21), # Martin Luther King Jr. day
            datetime(2019,2,18), # President's day
            datetime(2019,4,19), # Good Friday
            datetime(2019,5,27), # Memorial day
            datetime(2019,7,4), # Independence day
            datetime(2019,9,2), # Labor day
            datetime(2019,11,28), # Thanksgiving
            datetime(2019,12,25) # Christmas
]

In [111]:
# Japan holidays as of july 2009
Japan_holidays = [
            # https://www.sifma.org/resources/general/japan-holiday-archive/
            datetime(2009,1,1), # New Year's day
            datetime(2009,1,2), # Market Holiday
            datetime(2009,1,12), # Coming of Age Day / Adult's Day
            datetime(2009,2,11), # National Foundation Day
            datetime(2009,3,20), # Vernal Equinox
            datetime(2009,4,29), # Showa Day
            datetime(2009,5,4), # Greenery Day
            datetime(2009,5,5), # Children's Day
            datetime(2009,5,6), # Constitution Memorial Day
            datetime(2009,7,20), # Marine Day
            # Mountain Day - did not exist before 2014
            datetime(2009,9,21), # Respect for the Aged Day
            datetime(2009,9,22), 
            datetime(2009,9,23), # Autumnal Equinox
            datetime(2009,10,21), # Health and Sport's Day
            datetime(2009,11,3), # Culture Day
            datetime(2009,11,23), # Labor Thanksgiving Day
            datetime(2009,12,23), # Emperor's Birthday
            datetime(2009,12,31), # Market Holiday
    
            # https://www.sifma.org/resources/general/japan-holiday-archive/
            datetime(2010,1,1), # New Year's day
            datetime(2010,1,11), # Coming of Age Day / Adult's Day
            datetime(2010,2,11), # National Foundation Day
            datetime(2010,3,22), # Vernal Equinox
            datetime(2010,4,29), # Showa Day
            datetime(2010,5,3), # Constitution Memorial Day
            datetime(2010,5,4), # Greenery Day
            datetime(2010,5,5), # Children's Day
            datetime(2010,7,19), # Marine Day
            datetime(2010,9,20), # Respect for the Aged Day
            datetime(2010,9,23), # Autumnal Equinox
            datetime(2010,10,11), # Health and Sport's Day
            datetime(2010,11,3), # Culture Day
            datetime(2010,11,23), # Labor Thanksgiving Day
            datetime(2010,12,23), # Emperor's Birthday
#             datetime(2010,12,24), # Christmas ???
            datetime(2010,12,31), # Market Holiday
            
            # https://www.sifma.org/resources/general/japan-holiday-archive/
            datetime(2011,1,1), # New Year's day
            datetime(2011,1,3), # Market Holiday
            datetime(2011,1,10), # Coming of Age Day / Adult's Day
            datetime(2011,2,11), # National Foundation Day
            datetime(2011,3,21), # Vernal Equinox
            datetime(2011,4,29), # Showa Day
            datetime(2011,5,3), # Constitution Memorial Day
            datetime(2011,5,4), # Greenery Day
            datetime(2011,5,5), # Children's Day
            datetime(2011,7,18), # Marine Day
            datetime(2011,9,19), # Respect for the Aged Day
            datetime(2011,9,23), # Autumnal Equinox
            datetime(2011,10,10), # Health and Sport's Day
            datetime(2011,11,3), # Culture Day
            datetime(2011,11,23), # Labor Thanksgiving Day
            datetime(2011,12,23), # Emperor's Birthday
#             datetime(2011,12,26), # Christmas ???
            datetime(2011,12,31), # Market Holiday
    
            datetime(2012,1,1), # New Year's day
            datetime(2012,1,2), # Market Holiday
            datetime(2012,1,3), # Market Holiday
            datetime(2012,1,9), # Coming of Age Day / Adult's Day
#             datetime(2012,2,11), # National Foundation Day ????
            datetime(2012,3,20), # Vernal Equinox
            datetime(2012,4,30), # Showa Day
            datetime(2012,5,3), # Constitution Memorial Day
            datetime(2012,5,4), # Greenery Day
#             datetime(2012,5,5), # Children's Day
            datetime(2012,7,16), # Marine Day
            datetime(2012,9,17), # Respect for the Aged Day
#             datetime(2012,9,23), # Autumnal Equinox
            datetime(2012,10,8), # Health and Sport's Day
#             datetime(2012,11,3), # Culture Day
            datetime(2012,11,23), # Labor Thanksgiving Day
            datetime(2012,12,24), # Emperor's Birthday
#             datetime(2012,12,25), # Christmas ???
            datetime(2012,12,31), # Market Holiday
    
            datetime(2013,1,1), # New Year's day
            datetime(2013,1,2), # Market Holiday
            datetime(2013,1,3), # Market Holiday
            datetime(2013,1,14), # Coming of Age Day / Adult's Day
            datetime(2013,2,11), # National Foundation Day ????
            datetime(2013,3,20), # Vernal Equinox
            datetime(2013,4,29), # Showa Day
            datetime(2013,5,3), # Constitution Memorial Day
            datetime(2013,5,6), # Children's Day
            datetime(2013,7,15), # Marine Day
            datetime(2013,9,16), # Respect for the Aged Day
            datetime(2013,9,23), # Autumnal Equinox
            datetime(2013,10,14), # Health and Sport's Day
            datetime(2013,11,4), # Culture Day
            datetime(2013,12,23), # Emperor's Birthday
#             datetime(2013,12,25), # Christmas ???
            datetime(2013,12,31), # Market Holiday
    
            # https://stock-market-holidays.org/2014-tokyo-stock-exchange-holidays/
            datetime(2014,1,1), # New Year's day
            datetime(2014,1,2), # Market Holiday
            datetime(2014,1,3), # Market Holiday
            datetime(2014,1,13), # Coming of Age Day
            datetime(2014,2,11), # National Foundation Day
            datetime(2014,3,21), # Vernal Equinox
            datetime(2014,4,29), # Showa Day
            datetime(2014,5,3), # Constitution Memorial Day
            datetime(2014,5,4), # Greenery Day
            datetime(2014,5,5), # Children's Day
            datetime(2014,5,6), # Holiday
            datetime(2014,7,21), # Marine Day
            datetime(2014,9,15), # Respect for the Aged Day
            datetime(2014,9,22), # Holiday
            datetime(2014,9,23), # Autumnal Equinox
            datetime(2014,10,13), # Health and Sport's Day
            datetime(2014,11,3), # Culture Day
            datetime(2014,11,23), # Labor Thanksgiving Day
            datetime(2014,11,24), # Holiday
            datetime(2014,12,23), # Emperor's Birthday
            datetime(2014,12,31), # Market Holiday 
    
            # https://stock-market-holidays.org/2015-tokyo-stock-exchange-holidays/
            datetime(2015,1,1), # New Year's day
            datetime(2015,1,2), # Market Holiday
            datetime(2015,1,12), # Coming of Age Day
            datetime(2015,2,11), # National Foundation Day
            datetime(2015,3,21), # Vernal Equinox
            datetime(2015,4,29), # Showa Day
            datetime(2015,5,3), # Constitution Memorial Day
            datetime(2015,5,4), # Greenery Day
            datetime(2015,5,5), # Children's Day
            datetime(2015,5,6), # Holiday
            datetime(2015,7,20), # Marine Day
            datetime(2015,9,21), # Respect for the Aged Day
            datetime(2015,9,22), # Holiday
            datetime(2015,9,23), # Autumnal Equinox
            datetime(2015,10,12), # Health and Sport's Day
            datetime(2015,11,3), # Culture Day
            datetime(2015,11,23), # Labor Thanksgiving Day
            datetime(2015,12,23), # Emperor's Birthday
            datetime(2015,12,31), # Market Holiday
    
            # https://stock-market-holidays.org/2016-tokyo-stock-exchange-holidays/
            datetime(2016,1,1), # New Year's day
            datetime(2016,1,2), # Market Holiday
            datetime(2016,1,3), # Market Holiday
            datetime(2016,1,11), # Coming of Age Day
            datetime(2016,2,11), # National Foundation Day
            datetime(2016,3,21), # Vernal Equinox
            datetime(2016,4,29), # Showa Day
            datetime(2016,5,3), # Constitution Memorial Day
            datetime(2016,5,4), # Greenery Day
            datetime(2016,5,5), # Children's Day
            datetime(2016,7,18), # Marine Day
            datetime(2016,8,11), # Mountain Day
            datetime(2016,9,19), # Respect for the Aged Day
            datetime(2016,9,22), # Autumnal Equinox
            datetime(2016,10,10), # Health and Sport's Day
            datetime(2016,11,3), # Culture Day
            datetime(2016,11,23), # Labor Thanksgiving Day
            datetime(2016,12,23), # Emperor's Birthday
            datetime(2016,12,31), # Market Holiday
    
            # https://stock-market-holidays.org/2017-tokyo-stock-exchange-holidays/
            datetime(2017,1,1), # New Year's day
            datetime(2017,1,2), # Market Holiday
            datetime(2017,1,3), # Market Holiday
            datetime(2017,1,9), # Coming of Age Day
            datetime(2017,2,11), # National Foundation Day
            datetime(2017,3,20), # Vernal Equinox
            datetime(2017,4,29), # Showa Day
            datetime(2017,5,3), # Constitution Memorial Day
            datetime(2017,5,4), # Greenery Day
            datetime(2017,5,5), # Children's Day
            datetime(2017,7,17), # Marine Day
            datetime(2017,8,11), # Mountain Day
            datetime(2017,9,18), # Respect for the Aged Day
            datetime(2017,9,23), # Autumnal Equinox
            datetime(2017,10,9), # Health and Sport's Day
            datetime(2017,11,3), # Culture Day
            datetime(2017,11,23), # Labor Thanksgiving Day
            datetime(2017,12,23), # Emperor's Birthday
            datetime(2017,12,31), # Market Holiday
    
            # https://stock-market-holidays.org/2018-tokyo-stock-exchange-holidays/
            datetime(2018,1,1), # New Year's day
            datetime(2018,1,2), # Market Holiday
            datetime(2018,1,3), # Market Holiday
            datetime(2018,1,8), # Coming of Age Day
            datetime(2018,2,11), # National Foundation Day
            datetime(2018,2,12), # Holiday
            datetime(2018,3,21), # Vernal Equinox
            datetime(2018,4,29), # Showa Day
            datetime(2018,4,30), # Holiday
            datetime(2018,5,3), # Constitution Memorial Day
            datetime(2018,5,4), # Greenery Day
            datetime(2018,5,5), # Children's Day
            datetime(2018,7,16), # Marine Day
            datetime(2018,8,11), # Mountain Day
            datetime(2018,9,17), # Respect for the Aged Day
            datetime(2018,9,23), # Autumnal Equinox
            datetime(2018,9,24), # Holiday
            datetime(2018,10,8), # Health and Sport's Day
            datetime(2018,11,3), # Culture Day
            datetime(2018,11,23), # Labor Thanksgiving Day
            datetime(2018,12,23), # Emperor's Birthday
            datetime(2018,12,24), # Holiday
            datetime(2018,12,31), # Market Holiday
    
            # https://stock-market-holidays.org/2019-tokyo-stock-exchange-holidays/
            datetime(2019,1,1), # New Year's day
            datetime(2019,1,2), # Market Holiday
            datetime(2019,1,3), # Market Holiday
            datetime(2019,1,14), # Coming of Age Day
            datetime(2019,2,11), # National Foundation Day
            datetime(2019,3,21), # Vernal Equinox
            datetime(2019,4,29), # Showa Day
            datetime(2019,4,30), # stock market closure from 29-4 until 6-5
            datetime(2019,5,1),
            datetime(2019,5,2),
            datetime(2019,5,3), # Constitution Memorial Day
            datetime(2019,5,4), # Greenery Day
            datetime(2019,5,6), # Children's Day
            datetime(2019,7,15), # Marine Day
            datetime(2019,8,12), # Mountain Day
            datetime(2019,9,16), # Respect for the Aged Day
            datetime(2019,9,23), # Autumnal Equinox
            datetime(2019,10,14), # Health and Sport's Day
            datetime(2019,10,22),
            datetime(2019,11,4), # Culture Day
            datetime(2019,11,23), # Labor Thanksgiving Day
            datetime(2019,12,31) # Market Holiday
]

In [112]:
# German holidays

German_holidays = [
            # https://www.xetra.com/resource/blob/285992/ae41d04bf3c9163bf0190fdd43ed053e/data/trading-calendar-2009.pdf
            datetime(2009,1,1),
            datetime(2009,4,10),
            datetime(2009,4,13),
            datetime(2009,5,1),
            datetime(2009,12,24),
            datetime(2009,12,25),
            datetime(2009,12,31),
    
            # https://www.xetra.com/resource/blob/286014/22da6e6e049dfccee1e394cf2181b757/data/trading-calendar-2010.pdf
            datetime(2010,1,1),
            datetime(2010,4,2),
            datetime(2010,4,5),
            datetime(2010,12,24),
            datetime(2010,12,31),
    
            # https://www.xetra.com/resource/blob/285988/816cf6168a13ed9d2f06964a4ac29d1f/data/trading-calendar-2011.pdf
            datetime(2011,1,1),
            datetime(2011,4,22),
            datetime(2011,4,25),
            datetime(2011,12,26),  
    
            # https://www.xetra.com/resource/blob/281590/2420eeaa55be56d9f11a165184e34597/data/trading-calendar-2012.pdf
            datetime(2012,4,6),
            datetime(2012,4,9),
            datetime(2012,5,1), # labor day
            datetime(2012,12,24),
            datetime(2012,12,25),  
            datetime(2012,12,26),   
            datetime(2012,12,31),  
    
            # https://www.xetra.com/resource/blob/286022/49f43e63783f95e0039204ddf618e9ad/data/trading-calendar-2013.pdf
            datetime(2013,1,1),
            datetime(2013,3,29),
            datetime(2013,4,1),
            datetime(2013,5,1),  
            datetime(2013,12,24),
            datetime(2013,12,25),  
            datetime(2013,12,26),   
            datetime(2013,12,31),   

            # https://www.xetra.com/resource/blob/281576/967747fbc75fa120c2142e05d2e40711/data/trading-calendar-2014.pdf
            datetime(2014,1,1),
            datetime(2014,4,18), # Good Friday
            datetime(2014,4,21),
            datetime(2014,5,1),  
            datetime(2014,10,3),
            datetime(2014,12,24), 
            datetime(2014,12,25),  
            datetime(2014,12,26),   
            datetime(2014,12,31),   
    
            # https://www.xetra.com/resource/blob/252786/26a3eefdbb8b939be91ec1d3d1986347/data/trading-calendar-2015.pdf
            datetime(2015,1,1),
            datetime(2015,4,3), # Good Friday
            datetime(2015,4,6),
            datetime(2015,5,1), # labor day
            datetime(2015,5,25),  
            datetime(2015,12,24), 
            datetime(2015,12,25),    
            datetime(2015,12,31),  
    
            # https://www.xetra.com/resource/blob/259998/85c710e77ed5d595695f5b1d6418eac7/data/trading-calendar-2016.pdf
            datetime(2016,1,1),
            datetime(2016,3,25), # Good Friday
            datetime(2016,3,28),
            datetime(2016,5,16),  
            datetime(2016,10,3), 
            datetime(2016,12,26),    
            datetime(2016,12,31), 
            
            # https://www.xetra.com/resource/blob/336572/247517da046f56e5b4dae39723f29b00/data/trading-calendar-2017.pdf
            datetime(2017,4,14),
            datetime(2017,4,17),
            datetime(2017,5,1), # labor day
            datetime(2017,6,5),  
            datetime(2017,10,3),
            datetime(2017,10,31),
            datetime(2017,12,25),    
            datetime(2017,12,26), 
    
            # https://www.xetra.com/resource/blob/154430/366c1c9be4ce7cdb923bbf246db1d1bf/data/trading-calendar-2018.pdf
            datetime(2018,1,1),
            datetime(2018,3,30),
            datetime(2018,4,2),  
            datetime(2018,5,1),
            datetime(2018,5,21),
            datetime(2018,10,3),
            datetime(2018,12,24), 
            datetime(2018,12,25),    
            datetime(2018,12,26), 
            datetime(2018,12,31),
    
            # https://www.xetra.com/resource/blob/1406548/6de0eba301a5433abb110fa3c96a5778/data/xetra-trading-calendar-2019.pdf
            datetime(2019,1,1),
            datetime(2019,4,19),
            datetime(2019,4,22),  
            datetime(2019,5,1),
            datetime(2019,6,10),
            datetime(2019,10,3),
            datetime(2019,12,24), 
            datetime(2019,12,25),    
            datetime(2019,12,26), 
            datetime(2019,12,31),
]

In [113]:
UK_holidays = [
            # https://www.work-day.co.uk/workingdays_holidays_2011_London%20Stock%20Exchange.htm#showtext
            datetime(2009,1,1), # New Year's Day
            datetime(2009,4,10), # Good Friday
            datetime(2009,4,13), # Easter Monday
            datetime(2009,5,4), # Early May Bank Holiday
            datetime(2009,5,25), # Spring Bank Holiday
            datetime(2009,8,31), # Summer Bank Holiday  
#             datetime(2009,12,24), # Christmas Eve
            datetime(2009,12,25), # Christmas Day
            datetime(2009,12,28), # Boxing Day
            datetime(2009,12,31), # New Year's Eve
    
            # https://www.work-day.co.uk/workingdays_holidays_2011_London%20Stock%20Exchange.htm#showtext
            datetime(2010,1,1), # New Year's Day
            datetime(2010,4,2), # Good Friday
            datetime(2010,4,5), # Easter Monday
            datetime(2010,5,3), # Early May Bank Holiday
            datetime(2010,5,31), # Spring Bank Holiday
            datetime(2010,8,30), # Summer Bank Holiday  
            datetime(2010,12,24), # Christmas Eve
            datetime(2010,12,27), # Christmas Day
            datetime(2010,12,28), # Boxing Day
            datetime(2010,12,31), # New Year's Eve
    
            # https://www.work-day.co.uk/workingdays_holidays_2011_London%20Stock%20Exchange.htm#showtext
            datetime(2011,1,3), # New Year's Day
            datetime(2011,4,22), # Good Friday
            datetime(2011,4,25), # Easter Monday
            datetime(2011,4,29), # Royal Wedding
            datetime(2011,5,2), # Early May Bank Holiday
            datetime(2011,5,30), # Spring Bank Holiday
            datetime(2011,8,29), # Summer Bank Holiday  
            datetime(2011,12,26), # Christmas Day
            datetime(2011,12,27), # Boxing Day
    
            # https://holidaystracker.com/stock-market/london-stock-exchange-holidays-2012/
            datetime(2012,1,2), # New Year's Day
            datetime(2012,4,6), # Good Friday
            datetime(2012,4,9), # Easter Monday
            datetime(2012,5,7), # Early May Bank Holiday
            datetime(2012,6,4), # Spring Bank Holiday
            datetime(2012,6,5), # Queen's Diamond Jubilee
            datetime(2012,8,27), # Summer Bank Holiday  
            datetime(2012,12,25), # Christmas Day
            datetime(2012,12,26), # Boxing Day
    
            # https://holidaystracker.com/stock-market/london-stock-exchange-lse-holidays-2013/
            datetime(2013,1,1), # New Year's Day
            datetime(2013,3,29), # Good Friday
            datetime(2013,4,1), # Easter Monday
            datetime(2013,5,6), # Early May Bank Holiday
            datetime(2013,5,27), # Spring Bank Holiday
            datetime(2013,8,26), # Summer Bank Holiday  
            datetime(2013,12,25), # Christmas Day
            datetime(2013,12,26), # Boxing Day
    
            # https://stock-market-holidays.org/2014-london-stock-exchange-holidays/
            datetime(2014,1,1), # New Year's Day
            datetime(2014,3,29), # Good Friday
            datetime(2014,4,18), # Good Friday
            datetime(2014,4,21), # Easter Monday
            datetime(2014,5,2), # Early May Bank Holiday
            datetime(2014,5,5), # Early May Bank Holiday
            datetime(2014,5,26), # Spring Bank Holiday
            datetime(2014,8,25), # Summer Bank Holiday  
            datetime(2014,12,25), # Christmas Day
            datetime(2014,12,26), # Boxing Day
    
            # https://stock-market-holidays.org/2015-london-stock-exchange-holidays/
            datetime(2015,1,1), # New Year's Day
            datetime(2015,4,3), # Good Friday
            datetime(2015,4,6), # Easter Monday
            datetime(2015,5,4), # Early May Bank Holiday
            datetime(2015,5,25), # Spring Bank Holiday
            datetime(2015,8,31), # Summer Bank Holiday  
#             datetime(2015,12,24), Half day for Christmas Eve
            datetime(2015,12,25), # Christmas Day
            datetime(2015,12,28), # Boxing Day
#             datetime(2015,12,31) # Half day for New Year's Eve
    
            # https://stock-market-holidays.org/2016-london-stock-exchange-holidays/
            datetime(2016,1,1),
            datetime(2016,3,25),
            datetime(2016,3,28),  
            datetime(2016,5,2),
            datetime(2016,5,30),
            datetime(2016,8,29),   
            datetime(2016,12,26), 
            datetime(2016,12,27),
            datetime(2016,12,30),
    
            # https://stock-market-holidays.org/2017-london-stock-exchange-holidays/
            datetime(2017,1,2),
            datetime(2017,4,14),
            datetime(2017,4,17),  
            datetime(2017,5,1),
            datetime(2017,5,29),
            datetime(2017,8,28),   
            datetime(2017,12,25), 
            datetime(2017,12,26),
            datetime(2017,12,29),
    
            # https://stock-market-holidays.org/2018-london-stock-exchange-holidays/
            datetime(2018,1,1), # New Year's Day
            datetime(2018,3,30), # Good Friday
            datetime(2018,4,2), # Easter Monday
            datetime(2018,5,7), # Early May Bank Holiday
            datetime(2018,5,28), # Spring Bank Holiday
            datetime(2018,8,27), # Summer Bank Holiday
            datetime(2018,12,24), # Christmas Eve
            datetime(2018,12,25), # Christmas Day
            datetime(2018,12,26), # Boxing Day
            datetime(2018,12,31), # New Year's Eve
    
            # https://stock-market-holidays.org/2019-london-stock-exchange-holidays/
            datetime(2019,1,1), # New Year's Day
            datetime(2019,4,19), # Good Friday
            datetime(2019,4,22), # Easter Monday
            datetime(2019,5,6), # Early May Bank Holiday
            datetime(2019,5,27), # Spring Bank Holiday
            datetime(2019,8,26), # Summer Bank Holiday
            datetime(2019,12,24), # Christmas Eve
            datetime(2019,12,25), # Christmas Day
            datetime(2019,12,26), # Boxing Day
            datetime(2019,12,31) # New Year's Eve
]

In [114]:
Shanghai_holidays = [
            # https://www.china-briefing.com/news/china-announces-2009-holiday-schedule/
            datetime(2009,1,1), # New Year's Day
            datetime(2009,1,2), # New Year's Day
            datetime(2009,1,3), # New Year's Day
            datetime(2009,1,25), # First day of Lunar New Year
            datetime(2009,1,26), # Fourth day of Lunar New Year
            datetime(2009,1,27), # Fourth day of Lunar New Year
            datetime(2009,1,28), # Fourth day of Lunar New Year
            datetime(2009,1,29),
            datetime(2009,1,30),
            datetime(2009,1,31),
            datetime(2009,4,4),
            datetime(2009,4,5),
            datetime(2009,4,6),
            datetime(2009,5,1),
            datetime(2009,5,2),
            datetime(2009,5,3),
            datetime(2009,5,28),
            datetime(2009,5,29),
            datetime(2009,5,30),
            datetime(2009,10,1), # National day
            datetime(2009,10,2), # National day
            datetime(2009,10,3), # National day
            datetime(2009,10,4),
            datetime(2009,10,5),
            datetime(2009,10,6),
            datetime(2009,10,7),
            datetime(2009,10,8),
    
            # https://www.reuters.com/article/shanghai-copper-holiday-idUSTOE5BN06J20091224
            datetime(2010,1,1), # New Year's Day
            datetime(2010,2,15), # First day of Lunar New Year
            datetime(2010,2,16), # Fourth day of Lunar New Year
            datetime(2010,2,17), # Fourth day of Lunar New Year
            datetime(2010,2,18), # Fourth day of Lunar New Year
            datetime(2010,2,19),
            datetime(2010,4,5),
            datetime(2010,5,3),
            datetime(2010,6,14),
            datetime(2010,6,15),
            datetime(2010,6,16),
            datetime(2010,9,22),
            datetime(2010,9,23),
            datetime(2010,9,24),
            datetime(2010,10,1), # National day
            datetime(2010,10,2), # National day
            datetime(2010,10,3), # National day
            datetime(2010,10,4),
            datetime(2010,10,5),
            datetime(2010,10,6),
            datetime(2010,10,7),
    
            # https://www.china-briefing.com/news/china-announces-public-holiday-arrangement-for-2011/
            datetime(2011,1,1), # New Year's Day
            datetime(2011,1,2), # New Year's Day
            datetime(2011,1,3), # New Year's Day
            datetime(2011,2,2), # First day of Lunar New Year
            datetime(2011,2,3), # Fourth day of Lunar New Year
            datetime(2011,2,4), # Fourth day of Lunar New Year
            datetime(2011,2,5), # Fourth day of Lunar New Year
            datetime(2011,2,6),
            datetime(2011,2,7),
            datetime(2011,2,8),
            datetime(2011,4,3),
            datetime(2011,4,4),
            datetime(2011,4,5),
            datetime(2011,4,30),
            datetime(2011,5,1),
            datetime(2011,5,2),
            datetime(2011,6,4),
            datetime(2011,6,5),
            datetime(2011,6,6),
            datetime(2011,9,10),
            datetime(2011,9,11),
            datetime(2011,9,12),
            datetime(2011,10,1), # National day
            datetime(2011,10,2), # National day
            datetime(2011,10,3), # National day
            datetime(2011,10,4),
            datetime(2011,10,5),
            datetime(2011,10,6),
            datetime(2011,10,7),
    
            # https://holidaystracker.com/stock-market/shanghai-stock-exchange-sse-china-holidays-2013/
            datetime(2012,1,2), # New Year's Day
            datetime(2012,1,3),
            datetime(2012,1,23), # Lunar New Year's Eve
            datetime(2012,1,24), # First day of Lunar New Year
            datetime(2012,1,25), # Fourth day of Lunar New Year
            datetime(2012,1,26), # Fourth day of Lunar New Year
            datetime(2012,1,27), # Fourth day of Lunar New Year
            datetime(2012,4,2),
            datetime(2012,4,3),
            datetime(2012,4,4),
            datetime(2012,4,30),
            datetime(2012,5,1),
            datetime(2012,6,22),
            datetime(2012,10,1), # National day
            datetime(2012,10,2), # National day
            datetime(2012,10,3), # National day
            datetime(2012,10,4),
            datetime(2012,10,5),
    
            # https://holidaystracker.com/stock-market/shanghai-stock-exchange-sse-china-holidays-2013/
            datetime(2013,1,1), # New Year's Day
            datetime(2013,1,2), # New Year's Day
            datetime(2013,1,3), # New Year's Day
            datetime(2013,2,11), # Lunar New Year's Eve
            datetime(2013,2,12), # First day of Lunar New Year
            datetime(2013,2,13), # Fourth day of Lunar New Year
            datetime(2013,2,14), # Fourth day of Lunar New Year
            datetime(2013,2,15), # Fourth day of Lunar New Year
            datetime(2013,4,4),
            datetime(2013,4,5),
            datetime(2013,4,29),
            datetime(2013,4,30),
            datetime(2013,5,1),
            datetime(2013,6,10),
            datetime(2013,6,11),
            datetime(2013,6,12), # Ching Ming Festival
            datetime(2013,9,19),
            datetime(2013,9,20),
            datetime(2013,10,1), # National day
            datetime(2013,10,2), # National day
            datetime(2013,10,3), # National day
            datetime(2013,10,4),
    
            # http://www.shfe.com.cn/en/AnnouncementandNews/SHFEAnnouncement/911319055.html
            datetime(2014,1,1), # New Year's Day
            datetime(2014,1,31), # Lunar New Year's Eve
            datetime(2014,2,1), # First day of Lunar New Year
            datetime(2014,2,2), # Fourth day of Lunar New Year
            datetime(2014,2,3), # Fourth day of Lunar New Year
            datetime(2014,2,4), # Fourth day of Lunar New Year
            datetime(2014,2,5),
            datetime(2014,2,6),
            datetime(2014,4,5),
            datetime(2014,4,6),
            datetime(2014,4,7),
            datetime(2014,5,1),
            datetime(2014,5,2),
            datetime(2014,5,3),
            datetime(2014,5,31),
            datetime(2014,6,1),
            datetime(2014,6,2),
            datetime(2014,9,6),
            datetime(2014,9,7),
            datetime(2014,9,8),
            datetime(2014,10,1), # National day
            datetime(2014,10,2), # National day
            datetime(2014,10,3), # National day
            datetime(2014,10,4), # National day
            datetime(2014,10,5), # National day
            datetime(2014,10,6),
            datetime(2014,10,7),
    
            # https://stock-market-holidays.org/2015-shanghai-stock-exchange-holidays/
            datetime(2015,1,1), # New Year's Day
            datetime(2015,1,2), # New Year's Day
            datetime(2015,2,18), # Lunar New Year's Eve
            datetime(2015,2,19), # First day of Lunar New Year
            datetime(2015,2,20), # Fourth day of Lunar New Year
            datetime(2015,2,23), # Fourth day of Lunar New Year
            datetime(2015,2,24), # Fourth day of Lunar New Year
            datetime(2015,4,6),
            datetime(2015,5,1),
            datetime(2015,6,22), # Ching Ming Festival
            datetime(2015,9,3),
            datetime(2015,9,4),
            datetime(2015,10,1), # National day
            datetime(2015,10,2), # National day
            datetime(2015,10,5), # National day
            datetime(2015,10,6),
            datetime(2015,10,7),
    
            # https://www.china-briefing.com/news/chinas-2016-official-holiday-schedule-released/
            datetime(2016,1,1), # New Year's Day
            datetime(2016,1,2), # New Year's Day
            datetime(2016,1,3),
            datetime(2016,2,7),
            datetime(2016,2,8),
            datetime(2016,2,9), 
            datetime(2016,2,10),
            datetime(2016,2,11),
            datetime(2016,2,12),
            datetime(2016,2,13),
            datetime(2016,4,2),
            datetime(2016,4,3),
            datetime(2016,4,4),
            datetime(2016,4,30),
            datetime(2016,5,1),
            datetime(2016,5,2),
            datetime(2016,6,9), 
            datetime(2016,6,10), 
            datetime(2016,6,11), 
            datetime(2016,9,15), 
            datetime(2016,9,16), 
            datetime(2016,9,17), 
            datetime(2016,10,1), # National day
            datetime(2016,10,2), # National day
            datetime(2016,10,3), # National day
            datetime(2016,10,4),
            datetime(2016,10,5),
            datetime(2016,10,6),
            datetime(2016,10,7),
    
            # https://buysharesin.com/stock-market-holidays/
            datetime(2017,1,1), # New Year's Day
            datetime(2017,1,2), # New Year's Day
            datetime(2017,1,27), # Lunar New Year's Eve
            datetime(2017,1,28), # First day of Lunar New Year
            datetime(2017,1,29), # Fourth day of Lunar New Year
            datetime(2017,1,30), # Fourth day of Lunar New Year
            datetime(2017,1,31), # Fourth day of Lunar New Year
            datetime(2017,2,1),
            datetime(2017,2,2),
            datetime(2017,4,2), # Ching Ming Festival
            datetime(2017,4,3), # Ching Ming Festival Holiday
            datetime(2017,4,4),
            datetime(2017,4,29), # Labour day
            datetime(2017,4,30),
            datetime(2017,5,1), # Labour Day
            datetime(2017,5,28),
            datetime(2017,5,29),
            datetime(2017,5,30),
            datetime(2017,10,1), # National day
            datetime(2017,10,2), # National day
            datetime(2017,10,3), # National day
            datetime(2017,10,4), # National day
            datetime(2017,10,5), # National day
            datetime(2017,10,6),
            datetime(2017,10,7),
            datetime(2017,10,8),
        
            # https://stockmarket-holidays.com/shanghai-stock-exchange-holidays/17/
            datetime(2018,1,1), # New Year's Day
            datetime(2018,2,15), # Lunar New Year's Eve
            datetime(2018,2,16), # First day of Lunar New Year
            datetime(2018,2,19), # Fourth day of Lunar New Year
            datetime(2018,2,20), # Fourth day of Lunar New Year
            datetime(2018,2,21), # Fourth day of Lunar New Year
            datetime(2018,4,5), # Ching Ming Festival
            datetime(2018,4,6), # Ching Ming Festival Holiday
            datetime(2018,4,30), # Labour day
            datetime(2018,5,1), # Labour Day
            datetime(2018,6,18), # Tuen Ng Festival
            datetime(2018,9,24), # Chinese Mid-Autumn Festival
            datetime(2018,10,1), # National day
            datetime(2018,10,2), # National day
            datetime(2018,10,3), # National day
            datetime(2018,10,4), # National day
            datetime(2018,10,5), # National day
            datetime(2018,12,31), # New Year Holiday
    
            # https://stockmarket-holidays.com/shanghai-stock-exchange-holidays/17/
            datetime(2019,1,1), # New Year's Day
            datetime(2019,2,4), # Lunar New Year's Eve
            datetime(2019,2,5), # First day of Lunar New Year
            datetime(2019,2,6), # Second day of Lunar New Year
            datetime(2019,2,7), # Third day of Lunar New Year
            datetime(2019,2,8), # Fourth day of Lunar New Year
            datetime(2019,4,5), # Ching Ming Festival
            datetime(2019,5,1), # Labour Day
            datetime(2019,5,2), # Labour Day
            datetime(2019,5,3), # Labour Day
            datetime(2019,6,7), # Tuen Ng Festival
            datetime(2019,9,13), # Chinese Mid-Autumn Festival
            datetime(2019,10,1), # National day
            datetime(2019,10,2), # National day
            datetime(2019,10,3), # National day
            datetime(2019,10,4), # National day
            datetime(2019,10,7), # National day
]

HongKong_holidays = [
            # https://www.calendarlabs.com/holidays/hong-kong/2009
            datetime(2009,1,1),
            datetime(2009,2,13),
            datetime(2009,2,15),
            datetime(2009,2,16),
            datetime(2009,4,2),
            datetime(2009,4,3),
            datetime(2009,4,5),
            datetime(2009,4,6),
            datetime(2009,5,1),
            datetime(2009,5,21),
            datetime(2009,6,16),
            datetime(2009,7,1),
            datetime(2009,9,23),
            datetime(2009,10,1),
            datetime(2009,10,16),
            datetime(2009,12,25),
            datetime(2009,12,26),
    
            # https://www.calendarlabs.com/holidays/hong-kong/2010
            datetime(2010,1,1),
            datetime(2010,2,13),
            datetime(2010,2,15),
            datetime(2010,2,16),
            datetime(2010,4,2),
            datetime(2010,4,3),
            datetime(2010,4,5),
            datetime(2010,4,6),
            datetime(2010,5,1),
            datetime(2010,5,21),
            datetime(2010,6,16),
            datetime(2010,7,1),
            datetime(2010,9,23),
            datetime(2010,10,1),
            datetime(2010,10,16),
            datetime(2010,12,25),
            datetime(2010,12,27),
    
            # https://www.calendarlabs.com/holidays/hong-kong/2011
            datetime(2011,1,1),
            datetime(2011,2,3),
            datetime(2011,2,4),
            datetime(2011,2,5),
            datetime(2011,4,5),
            datetime(2011,4,22),
            datetime(2011,4,23),
            datetime(2011,4,25),
            datetime(2011,5,1),
            datetime(2011,5,2),
            datetime(2011,5,10),
            datetime(2011,6,6),
            datetime(2011,7,1),
            datetime(2011,9,13),
            datetime(2011,10,1),
            datetime(2011,10,5),
            datetime(2011,12,25),
            datetime(2011,12,26),
            datetime(2011,12,27),
    
            # https://holidaystracker.com/stock-market/hong-kong-stock-exchange-holidays-2012/
            datetime(2012,1,2),
            datetime(2012,1,23),
            datetime(2012,1,24),
            datetime(2012,1,25),
            datetime(2012,4,4),
            datetime(2012,4,6),
            datetime(2012,4,9),
            datetime(2012,5,1),
            datetime(2012,7,2),
            datetime(2012,10,1),
            datetime(2012,10,2),
            datetime(2012,10,23),
            datetime(2012,12,25),
            datetime(2012,12,26),
    
            # https://holidaystracker.com/stock-market/hong-kong-stock-exchange-holidays-2013/
            datetime(2013,1,1),
            datetime(2013,2,11),
            datetime(2013,2,12),
            datetime(2013,2,13),
            datetime(2013,3,29),
            datetime(2013,4,1),
            datetime(2013,4,4),
            datetime(2013,5,1),
            datetime(2013,5,17),
            datetime(2013,6,12),
            datetime(2013,7,1),
            datetime(2013,9,20),
            datetime(2013,10,1),
            datetime(2013,10,14),
            datetime(2013,12,25),
            datetime(2013,12,26),
    
            # https://stock-market-holidays.org/2014-hong-kong-stock-exchange-holidays/
            datetime(2014,1,1),
            datetime(2014,1,31),
            datetime(2014,2,3),
            datetime(2014,4,18),
            datetime(2014,4,21),
            datetime(2014,5,1),
            datetime(2014,5,6),
            datetime(2014,6,2),
            datetime(2014,7,1),
            datetime(2014,9,9),
            datetime(2014,10,1),
            datetime(2014,10,2),
            datetime(2014,12,25),
            datetime(2014,12,26),
    
            # https://stock-market-holidays.org/2015-hong-kong-stock-exchange-holidays/
            datetime(2015,1,1), # New Year's Day
            datetime(2015,2,18), # First day of Lunar New Year
            datetime(2015,2,19), # Second day of Lunar New Year
            datetime(2015,2,20), # Third day of Lunar New Year
            datetime(2015,4,3), # Good Friday
            datetime(2015,4,6), # Day Following Ching Ming Festival
            datetime(2015,4,7), # Day Following Easter Monday
            datetime(2015,5,1), # Labor Day
            datetime(2015,5,25), # Birthday of Buddha
            datetime(2015,7,1), # Hong Kong Special Administrative Region Establishment Day
            datetime(2015,9,3),
            datetime(2015,9,28), # Day following Chinese Mid-Autumn Festival
            datetime(2015,10,1), # National day
            datetime(2015,10,21), # Chung Yueng Festival
#             datetime(2015,12,24), # half day for christmas eve
            datetime(2015,12,25), # Christmas day
#             datetime(2015,12,31), # half day for new year's eve
        
            # https://stock-market-holidays.org/2016-hong-kong-stock-exchange-holidays/
            datetime(2016,1,1), # New Year's Day
            datetime(2016,2,8), # First day of Lunar New Year
            datetime(2016,2,9), # Second day of Lunar New Year
            datetime(2016,2,10), # Third day of Lunar New Year
            datetime(2016,3,25), # Good Friday
            datetime(2016,3,28), # Easter Monday
            datetime(2016,4,4), # Ching Ming Festival
            datetime(2016,5,2), # Day Following Labour day
            datetime(2016,6,9), # Tuen Ng Festival
            datetime(2016,7,1), # Hong Kong Special Administrative Region Establishment Day
            datetime(2016,8,2),
            datetime(2016,9,16), # Day following Chinese Mid-Autumn Festival
            datetime(2016,10,10), # Day Following Chung Yueng Festival
            datetime(2016,12,25), # Christmas day
            datetime(2016,12,26), # Boxing day
            datetime(2016,12,27), # Boxing day
    
            # https://stock-market-holidays.org/2017-hong-kong-stock-exchange-holidays/
            datetime(2017,1,2), # New Year's Day
            datetime(2017,1,30), # Third day of Lunar New Year
            datetime(2017,1,31), # Fourth day of Lunar New Year
            datetime(2017,4,4), # Ching Ming Festival
            datetime(2017,4,14), # Good Friday
            datetime(2017,4,17), # Easter Monday
            datetime(2017,5,1), # Labour day
            datetime(2017,5,3), # Birthday of the Buddha
            datetime(2017,5,30), # Tuen Ng Festival
            datetime(2017,10,2), # Day Following National Day
            datetime(2017,10,5), # Day following Chinese Mid-Autumn Festival
            datetime(2017,12,25), # Christmas day
            datetime(2017,12,26), # Boxing day
    
            # https://stock-market-holidays.org/2018-hong-kong-stock-exchange-holidays/
            datetime(2018,1,1), # New Year's Day
            datetime(2018,2,16), # First day of Lunar New Year
            datetime(2018,2,19), # Fourth day of Lunar New Year
            datetime(2018,3,30), # Good Friday
            datetime(2018,4,2), # Easter Monday
            datetime(2018,4,5), # Ching Ming Festival
            datetime(2018,5,1), # Labour day
            datetime(2018,5,22), # Birthday of the Buddha
            datetime(2018,6,18), # Tuen Ng Festival
            datetime(2018,7,2), # Hong Kong Special Administrative Region Establishment Day
            datetime(2018,9,25), # Day following Chinese Mid-Autumn Festival
            datetime(2018,10,1), # National day
            datetime(2018,10,17), # Chung Yueng Festival
            datetime(2018,12,25), # Christmas day
            datetime(2018,12,26), # Boxing day
    
            # https://stock-market-holidays.org/2019-hong-kong-stock-exchange-holidays/
            datetime(2019,1,1), # New Year's Day
            datetime(2019,2,5), # First day of Lunar New Year
            datetime(2019,2,6), # Second day of Lunar New Year
            datetime(2019,2,7), # Third day of Lunar New Year
            datetime(2019,4,5), # Ching Ming Festival
            datetime(2019,4,19), # Good Friday
            datetime(2019,4,20), # Day following Good Friday
            datetime(2019,4,22), # Easter Monday
            datetime(2019,5,1), # Labour day
            datetime(2019,5,13), # Day following Birthday of the Buddha
            datetime(2019,6,7), # Tuen Ng Festival
            datetime(2019,7,1), # Hong Kong Special Administrative Region Establishment Day
            datetime(2019,9,14), # Day following Chinese Mid-Autumn Festival
            datetime(2019,10,1), # National day
            datetime(2019,10,7), # Chung Yueng Festival
            datetime(2019,12,25), # Christmas day
            datetime(2019,12,26), # Boxing day
]

In [115]:
France_holidays = [
            # https://www.reuters.com/article/diary-european-holidays-idUSL0613093820090724
            datetime(2009,12,25), # Christmas Eve
    
            # https://www.reuters.com/article/diary-european-holidays-idUSL0613093820091203
            datetime(2010,1,1), # New Year's Day
            datetime(2010,4,2), # Good Friday
            datetime(2010,4,5), # Easter Monday
            datetime(2010,12,25), # Christmas Eve
    
            # https://www.cmvm.pt/pt/SDI/Mercados/Documents/Calendário%20Euronext%202011.pdf
            datetime(2011,4,22), # Good Friday
            datetime(2011,4,25), # Easter Monday
            datetime(2011,12,26), # Christmas Day
    
            # https://www.euronext.com/en/about/media/euronext-press-releases/nyse-euronext-announces-its-2012-holiday-calendar-and-early
            datetime(2012,4,6), # Good Friday
            datetime(2012,4,9), # Easter Monday
            datetime(2012,5,1), # Labor Day
            datetime(2012,12,25), # Christmas Eve
            datetime(2012,12,26), # Christmas Day
    
            # https://www.euronext.com/en/about/media/euronext-press-releases/nyse-euronext-announces-its-2013-holiday-calendar-and-early
            datetime(2013,1,1), # New Year's Day
            datetime(2013,3,29), # Good Friday
            datetime(2013,4,1), # Easter Monday
            datetime(2013,5,1), # Labor Day
            datetime(2013,12,25), # Christmas Eve
            datetime(2013,12,26), # Christmas Day
    
            # https://www.euronext.com/en/about/media/euronext-press-releases/nyse-euronext-announces-its-2014-holiday-calendar-and-early
            datetime(2014,1,1), # New Year's Day
            datetime(2014,4,18), # Good Friday
            datetime(2014,4,21), # Easter Monday
            datetime(2014,5,1), # Labor Day
            datetime(2014,12,25), # Christmas Eve
            datetime(2014,12,26), # Christmas Day
    
            # https://stock-market-holidays.org/2015-euronext-stock-exchange-holidays/
            datetime(2015,1,1), # New Year's Day
            datetime(2015,4,3), # Good Friday
            datetime(2015,4,6), # Easter Monday
            datetime(2015,5,1), # Labor Day
            datetime(2015,12,25), # Christmas Eve
            datetime(2015,12,31), # New Year's Eve
    
            # https://www.euronext.com/en/about/media/euronext-press-releases/euronext-announces-calendar-2016
            datetime(2016,1,1), # New Year's Day
            datetime(2016,3,25), # Good Friday
            datetime(2016,3,28), # Easter Monday
            datetime(2016,12,26), # Christmas Day
    
            # https://stock-market-holidays.org/2017-euronext-stock-exchange-holidays/
            datetime(2017,4,14), # Good Friday
            datetime(2017,4,17), # Easter Monday
            datetime(2017,5,1), # Labor Day
            datetime(2017,12,25), # Christmas Eve
            datetime(2017,12,26), # Christmas Day
    
            # https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&ved=2ahUKEwjq9862uO32AhWE_6QKHd1iCZEQFnoECAQQAQ&url=https%3A%2F%2Fwww.euronext.com%2Fsites%2Fdefault%2Ffiles%2F17.10.20.holiday_calendar_2018.pdf&usg=AOvVaw3uWWSCUagWS7R2MUHDAg00
            datetime(2018,1,1), # New Year's Day
            datetime(2018,3,30), # Good Friday
            datetime(2018,4,2), # Easter Monday
            datetime(2018,5,1), # Labor Day
            datetime(2018,12,25), # Christmas Eve
            datetime(2018,12,26), # Christmas Day

            # https://www.stockmaster.in/howtoinvest/cac-40-euronext-paris-stock-market-holiday-calendar-2019/
            datetime(2019,1,1), # New Year's Day
            datetime(2019,4,19), # Good Friday
            datetime(2019,4,22), # Easter Monday
            datetime(2019,5,1), # Labor Day
            datetime(2019,12,25), # Christmas Eve
            datetime(2019,12,26) # Christmas Day
]

In [116]:
def find_missing_days(df, varname):
    sdate = datetime(2009,7,1)
    edate = datetime(2020,1,1)
    # make date range between start and end dates
    list_dates = [sdate + timedelta(days=x) for x in range((edate - sdate).days)]
    # remove weekends from this list as weekends are not trading days
    new_list = []
    exclude = [5, 6]
    for date in list_dates:
        if date.weekday() not in exclude:
            new_list.append(date)
    
    # determine which weekdays are not present in data
    for i, row in df.iterrows():
        if row['Date'] in new_list:
            new_list.remove(row['Date'])

    # determine which absent weekdays are not holidays
    missing_days = []
    if varname == "S&P500" or varname == "NASDAQ" or varname == "US30" or varname == "US2000":
        for i in new_list:
            if i not in US_holidays:
                missing_days.append(i)
    elif varname == "GER30":
        for i in new_list:
            if i not in German_holidays:
                missing_days.append(i)    
    elif varname == "UK100":
        for i in new_list:
            if i not in UK_holidays:
                missing_days.append(i)  
    elif varname == "CAC40":
        for i in new_list:
            if i not in France_holidays:
                missing_days.append(i)  
    elif varname == "NIKKEI225":
        for i in new_list:
            if i not in Japan_holidays:
                missing_days.append(i)
    elif varname == "HS50":
        for i in new_list:
            if i not in HongKong_holidays:
                missing_days.append(i)
    elif varname == "SSE50":
        for i in new_list:
            if i not in Shanghai_holidays:
                missing_days.append(i)
    print(f"{varname}\tMissing dates: {len(missing_days)}")
    for i in missing_days:
        print(f"\t{i.date()}")
    
files = {
    # varname: filename
    "S&P500": "Dataset v2/Indices/S&P 500 Historical Data.csv",
    "US30": "Dataset v2/Indices/Dow Jones Industrial Average Historical Data.csv", 
    "US2000": "Dataset v2/Indices/US Small Cap 2000 Historical Data.csv",
    "NASDAQ": "Dataset v2/Indices/NASDAQ Composite Historical Data.csv", 
    "GER30": "Dataset v2/Indices/DAX Historical Data.csv",
    "CAC40": "Dataset v2/Indices/CAC 40 Historical Data.csv",
    "UK100": "Dataset v2/Indices/invUK100 Historical Data.csv",
    "SSE50": "Dataset v2/Indices/Shanghai Composite Historical Data.csv",
    "HS50": "Dataset v2/Indices/Hang Seng Historical Data.csv",
    "NIKKEI225": "Dataset v2/Indices/NIKKEI 225 Historical Data.csv",
}

print(f"Retrieving {len(files)} files")
i = 1
df = pd.DataFrame()
stats_rows = []
for file in files:
    df = retrieve_data(files[file], file)
    find_missing_days(df, file)

Retrieving 10 files
S&P500	Missing dates: 2
	2012-10-29
	2012-10-30
US30	Missing dates: 2
	2012-10-29
	2012-10-30
US2000	Missing dates: 2
	2012-10-29
	2012-10-30
NASDAQ	Missing dates: 2
	2012-10-29
	2012-10-30
GER30	Missing dates: 0
CAC40	Missing dates: 0
UK100	Missing dates: 2
	2010-07-28
	2019-03-05
SSE50	Missing dates: 1
	2013-10-07
HS50	Missing dates: 3
	2011-09-29
	2013-08-14
	2016-10-21
NIKKEI225	Missing dates: 3
	2009-10-12
	2010-07-28
	2010-07-29
